In [2]:
import pandas as pd 
from sqlalchemy import create_engine
server = 'rice-sqlcl-db'
database = 'SharadarEquity'
username = 'stocks'
password = '6LAZH1'
string = f"mssql+pymssql://{username}:{password}@{server}/{database}"
conn = create_engine(string).connect() 


In [3]:
df.date.unique()

array([datetime.date(2025, 3, 21)], dtype=object)

In [3]:
import pandas as pd 
df = pd.read_sql("select * from information_schema.tables", conn)

In [4]:
df

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,SharadarEquity,dbo,today,BASE TABLE
1,SharadarEquity,dbo,ghz,BASE TABLE
2,SharadarEquity,dbo,definitions,BASE TABLE
3,SharadarEquity,dbo,tickers,BASE TABLE
4,SharadarEquity,dbo,prices,BASE TABLE
5,SharadarEquity,dbo,fundamentals,BASE TABLE
6,SharadarEquity,dbo,metrics,BASE TABLE


In [26]:
for col in df.columns: print(col)


ticker
siccode
name
sicsector
sicindustry
famasector
famaindustry
sector
industry
scalemarketcap
scalerevenue
dimension
fiscalperiod
calendardate
datekey
reportperiod
lastupdated
accoci
assets
assetsavg
assetsc
assetsnc
assetturnover
bvps
capex
cashneq
cashnequsd
cor
consolinc
currentratio
de
debt
debtc
debtnc
debtusd
deferredrev
depamor
deposits
divyield
dps
ebit
ebitda
ebitdamargin
ebitdausd
ebitusd
ebt
eps
epsdil
epsusd
equity
equityavg
equityusd
ev
evebit
evebitda
fcf
fcfps
fxusd
gp
grossmargin
intangibles
intexp
invcap
invcapavg
inventory
investments
investmentsc
investmentsnc
liabilities
liabilitiesc
liabilitiesnc
marketcap
ncf
ncfbus
ncfcommon
ncfdebt
ncfdiv
ncff
ncfi
ncfinv
ncfo
ncfx
netinc
netinccmn
netinccmnusd
netincdis
netincnci
netmargin
opex
opinc
payables
payoutratio
pb
pe
pe1
ppnenet
prefdivis
price
ps
ps1
receivables
retearn
revenue
revenueusd
rnd
roa
roe
roic
ros
sbcomp
sgna
sharefactor
sharesbas
shareswa
shareswadil
sps
tangibles
taxassets
taxexp
taxliabilities
tbvps

In [30]:
pd.read_sql("select * from definitions where 1=2", conn)

,tbl,indicator,isfilter,isprimarykey,title,description,unittype


In [33]:
indicators = pd.read_sql("select * from definitions where tbl='SF1'", conn)
for row in indicators.index: print(indicators[["indicator", "description"]].loc[row])

indicator                                                revenue
description    [Income Statement] The amount of Revenue recog...
Name: 0, dtype: object
indicator                                                    cor
description    [Income Statement] The aggregate cost of goods...
Name: 1, dtype: object
indicator                                                   sgna
description    [Income Statement] A component of [OpEx] repre...
Name: 2, dtype: object
indicator                                                    rnd
description    [Income Statement] A component of [OpEx] repre...
Name: 3, dtype: object
indicator                                                   opex
description    [Income Statement] Operating expenses represen...
Name: 4, dtype: object
indicator                                                 intexp
description    [Income Statement] Amount of the cost of borro...
Name: 5, dtype: object
indicator                                                 taxexp
description    [I

In [34]:
financials="eps fcfps roe roa grossmargin netmargin ebitdamargin ros assetturnover payoutratio evebitda evebit pe ps pb de divyield currentratio assets debt liabilities equity revenue".split()

In [40]:
df = pd.read_sql("select * from fundamentals where dimension='ARY' and reportperiod>='2024-01-01' and reportperiod<='2025-12-31'", conn)

In [41]:
df=df[["ticker","reportperiod"]+financials]

In [42]:
df2 = pd.read_sql("select * from tickers", conn)
df2 = df2[
    [
        "ticker", "name", 
        "sector", "industry", "scalemarketcap", "scalerevenue"
    ]
]

In [43]:

df = df2.merge(df, left_on="ticker", right_on="ticker", how="inner")

In [45]:
df=df.dropna()

In [46]:
df.head()

,ticker,name,sector,industry,scalemarketcap,scalerevenue,reportperiod,eps,fcfps,roe,...,ps,pb,de,divyield,currentratio,assets,debt,liabilities,equity,revenue
0,A,AGILENT TECHNOLOGIES INC,Healthcare,Diagnostics & Research,5 - Large,5 - Large,2024-10-31,4.44,4.734,0.213,...,5.901,6.513,1.008,0.007,2.089,1.184600e+10,3.390000e+09,5.948000e+09,5.898000e+09,6.510000e+09
1,AA,ALCOA CORP,Basic Materials,Aluminum,5 - Large,5 - Large,2024-12-31,0.26,0.162,0.013,...,0.811,1.870,1.727,0.011,1.447,1.406400e+10,2.545000e+09,8.907000e+09,5.157000e+09,1.189500e+10
4,AAL,AMERICAN AIRLINES GROUP INC,Industrials,Airlines,5 - Large,5 - Large,2024-12-31,1.29,2.972,-0.177,...,0.194,-2.646,-16.535,0.000,0.541,6.178300e+10,3.754400e+10,6.576000e+10,-3.977000e+09,5.421100e+10
8,AAOI,APPLIED OPTOELECTRONICS INC,Technology,Communication Equipment,3 - Small,3 - Small,2024-12-31,-4.50,-2.882,-0.907,...,4.353,4.737,1.388,0.000,1.771,5.470320e+08,1.908600e+08,3.179200e+08,2.291120e+08,2.493650e+08
9,AAON,AAON INC,Basic Materials,Building Products & Equipment,4 - Mid,3 - Small,2024-12-31,2.07,-0.038,0.214,...,5.345,7.782,0.425,0.004,2.791,1.175234e+09,1.548910e+08,3.506520e+08,8.245820e+08,1.200635e+09


In [ ]:
df.to_excel("financials.xlsx")

In [12]:
df1[df1.indicator.isin(["datekey", "calendardate", "reportperiod"])]

,tbl,indicator,isfilter,isprimarykey,title,description,unittype
107,SF1,datekey,N,Y,Date Key,[Entity] The Date Key represents the SEC filin...,date (YYYY-MM-DD)
108,SF1,reportperiod,N,Y,Report Period,[Entity] The Report Period represents the end ...,date (YYYY-MM-DD)
110,SF1,calendardate,Y,N,Calendar Date,[Entity] The Calendar Date represents the norm...,date (YYYY-MM-DD)
139,SF3,calendardate,Y,Y,Calendar Date,The calendar date field represents the last da...,date (YYYY-MM-DD)
143,SF3A,calendardate,Y,Y,Calendar Date,The calendar date field represents the last da...,date (YYYY-MM-DD)
172,SF3B,calendardate,Y,Y,Calendar Date,The calendar date field represents the last da...,date (YYYY-MM-DD)
